# Week 12: Backtesting, Transaction Costs, Walk-Forward Validation

## 🎯 Learning Objectives

By the end of this week, you will understand:
- **Backtesting**: Simulating strategy performance
- **Transaction Costs**: Slippage, commissions, market impact
- **Walk-Forward Validation**: Proper train/test for time series
- **Performance Metrics**: Sharpe, Sortino, Max Drawdown

---

## Why Proper Backtesting?

Most backtested strategies fail in live trading because of:
- **Lookahead bias**: Using future information
- **Survivorship bias**: Only testing on surviving stocks
- **Overfitting**: Fitting noise, not signal
- **Ignoring costs**: Transaction costs eat profits

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
print("✅ Libraries loaded!")
print("📚 Week 12: Backtesting & Validation")

---

## Part 1: Performance Metrics

### Key Metrics

**Sharpe Ratio**: Risk-adjusted return
$$SR = \frac{E[R - R_f]}{\sigma_R} \times \sqrt{252}$$

**Sortino Ratio**: Only penalizes downside
$$Sortino = \frac{E[R - R_f]}{\sigma_{downside}} \times \sqrt{252}$$

**Maximum Drawdown**: Largest peak-to-trough decline
$$MDD = \max_t \left(\frac{\text{Peak}_t - \text{Value}_t}{\text{Peak}_t}\right)$$

In [ ]:
def calculate_metrics(returns):
    """Calculate key performance metrics"""
    returns = pd.Series(returns)
    
    # Annualized return
    total_return = (1 + returns).prod() - 1
    n_years = len(returns) / 252
    annual_return = (1 + total_return) ** (1/n_years) - 1
    
    # Sharpe Ratio
    sharpe = returns.mean() / returns.std() * np.sqrt(252)
    
    # Sortino Ratio
    downside = returns[returns < 0].std()
    sortino = returns.mean() / downside * np.sqrt(252) if downside > 0 else np.inf
    
    # Max Drawdown
    cumulative = (1 + returns).cumprod()
    peak = cumulative.expanding().max()
    drawdown = (cumulative - peak) / peak
    max_dd = drawdown.min()
    
    return {
        'Total Return': f"{total_return:.2%}",
        'Annual Return': f"{annual_return:.2%}",
        'Annual Vol': f"{returns.std() * np.sqrt(252):.2%}",
        'Sharpe Ratio': f"{sharpe:.2f}",
        'Sortino Ratio': f"{sortino:.2f}",
        'Max Drawdown': f"{max_dd:.2%}",
    }

# Example
returns_example = np.random.randn(252) * 0.01 + 0.0003
metrics = calculate_metrics(returns_example)

print("Performance Metrics Example")
print("="*50)
for k, v in metrics.items():
    print(f"{k:20}: {v}")

---

## Part 2: Transaction Costs

### Cost Components

1. **Commission**: Fixed fee per trade
2. **Bid-Ask Spread**: Cost of crossing the spread
3. **Slippage**: Price movement during execution
4. **Market Impact**: Large orders move prices

### Rule of Thumb

- Liquid stocks: 5-10 bps per trade
- Illiquid stocks: 20-50+ bps per trade
- High frequency: Costs dominate

In [ ]:
def backtest_with_costs(signals, returns, cost_bps=10):
    """Backtest with transaction costs"""
    cost = cost_bps / 10000  # Convert bps to decimal
    
    # Calculate turnover (change in position)
    turnover = np.abs(np.diff(signals, prepend=0))
    
    # Gross returns (before costs)
    gross_returns = signals * returns
    
    # Transaction costs
    costs = turnover * cost
    
    # Net returns
    net_returns = gross_returns - costs
    
    return gross_returns, net_returns, costs

# Simulate a momentum strategy
n = 252 * 2  # 2 years
returns_sim = np.random.randn(n) * 0.01 + 0.0002

# Signal based on rolling momentum
momentum = pd.Series(returns_sim).rolling(20).mean()
signal = np.where(momentum > 0, 1, -1)
signal = np.nan_to_num(signal)

# Backtest with different cost assumptions
costs_bps = [0, 5, 10, 20, 50]

print("Impact of Transaction Costs")
print("="*60)
print(f"{'Cost (bps)':<15} {'Sharpe':<12} {'Total Return':<15} {'Turnover'}")
print("-"*60)

for cost in costs_bps:
    gross, net, _ = backtest_with_costs(signal, returns_sim, cost)
    sharpe = pd.Series(net).mean() / pd.Series(net).std() * np.sqrt(252)
    total_ret = (1 + pd.Series(net)).prod() - 1
    turnover = np.abs(np.diff(signal)).sum()
    print(f"{cost:<15} {sharpe:<12.2f} {total_ret:<15.2%} {turnover}")

In [ ]:
# Visualize cost impact
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Cumulative returns
for cost in [0, 10, 30]:
    _, net, _ = backtest_with_costs(signal, returns_sim, cost)
    cum = (1 + pd.Series(net)).cumprod()
    axes[0].plot(cum, label=f'{cost} bps')

axes[0].set_xlabel('Days')
axes[0].set_ylabel('Cumulative Return')
axes[0].set_title('Impact of Transaction Costs')
axes[0].legend()

# Sharpe degradation
sharpes = []
costs_range = range(0, 51, 5)
for cost in costs_range:
    _, net, _ = backtest_with_costs(signal, returns_sim, cost)
    sharpe = pd.Series(net).mean() / pd.Series(net).std() * np.sqrt(252)
    sharpes.append(sharpe)

axes[1].plot(costs_range, sharpes, 'bo-')
axes[1].axhline(y=0, color='r', linestyle='--')
axes[1].set_xlabel('Transaction Cost (bps)')
axes[1].set_ylabel('Sharpe Ratio')
axes[1].set_title('Sharpe Degradation with Costs')

plt.tight_layout()
plt.show()

---

## Part 3: Walk-Forward Validation

### The Problem with Standard CV

K-Fold CV shuffles data → destroys temporal structure → causes lookahead bias!

### Walk-Forward Solution

1. Train on historical data
2. Test on next period
3. Roll forward
4. Repeat

```
Train: |--------|          Test: |--|
Train:    |--------|       Test:    |--|
Train:       |--------|    Test:       |--|
```

In [ ]:
def walk_forward_cv(X, y, model, train_size=252, test_size=21, gap=0):
    """Walk-forward cross-validation"""
    n = len(X)
    predictions = []
    actuals = []
    dates = []
    
    start = train_size
    while start + test_size <= n:
        # Define train/test splits
        train_end = start
        test_start = start + gap
        test_end = test_start + test_size
        
        if test_end > n:
            break
            
        # Train
        X_train = X[:train_end]
        y_train = y[:train_end]
        
        # Test
        X_test = X[test_start:test_end]
        y_test = y[test_start:test_end]
        
        # Fit and predict
        model.fit(X_train, y_train)
        preds = model.predict(X_test)
        
        predictions.extend(preds)
        actuals.extend(y_test)
        dates.extend(range(test_start, test_end))
        
        # Roll forward
        start += test_size
    
    return np.array(predictions), np.array(actuals), np.array(dates)

In [ ]:
# Create features and target
n = 252 * 3  # 3 years
returns_ts = np.random.randn(n) * 0.01 + 0.0001

# Features: lagged returns and volatility
df_ts = pd.DataFrame({'return': returns_ts})
df_ts['lag1'] = df_ts['return'].shift(1)
df_ts['lag5'] = df_ts['return'].shift(5)
df_ts['vol20'] = df_ts['return'].rolling(20).std()
df_ts['target'] = df_ts['return'].shift(-1)  # Next day return
df_ts = df_ts.dropna()

X_ts = df_ts[['lag1', 'lag5', 'vol20']].values
y_ts = df_ts['target'].values

# Compare standard CV vs walk-forward
from sklearn.model_selection import cross_val_score, TimeSeriesSplit

model = Ridge(alpha=1.0)

# Standard K-Fold (WRONG for time series)
kfold_scores = cross_val_score(model, X_ts, y_ts, cv=5)

# Time Series Split (correct)
tscv = TimeSeriesSplit(n_splits=5)
tscv_scores = cross_val_score(model, X_ts, y_ts, cv=tscv)

print("Cross-Validation Comparison")
print("="*50)
print(f"Standard K-Fold: {kfold_scores.mean():.4f} ± {kfold_scores.std():.4f}")
print(f"Time Series CV:  {tscv_scores.mean():.4f} ± {tscv_scores.std():.4f}")
print(f"\n⚠️ K-Fold often overstates performance due to lookahead bias!")

In [ ]:
# Walk-forward backtest
preds, actuals, dates = walk_forward_cv(
    X_ts, y_ts, 
    model=Ridge(alpha=1.0),
    train_size=252,
    test_size=21
)

# Convert predictions to trading signals
signals_wf = np.sign(preds)
strategy_returns = signals_wf * actuals

# Calculate metrics
metrics_wf = calculate_metrics(strategy_returns)

print("\nWalk-Forward Strategy Performance")
print("="*50)
for k, v in metrics_wf.items():
    print(f"{k:20}: {v}")

# Visualize
fig, axes = plt.subplots(2, 1, figsize=(12, 8))

# Cumulative returns
cum_strat = (1 + pd.Series(strategy_returns)).cumprod()
cum_bh = (1 + pd.Series(actuals)).cumprod()
axes[0].plot(cum_strat.values, label='Strategy')
axes[0].plot(cum_bh.values, label='Buy & Hold', alpha=0.7)
axes[0].set_xlabel('Days')
axes[0].set_ylabel('Cumulative Return')
axes[0].set_title('Walk-Forward Backtest')
axes[0].legend()

# Rolling Sharpe
rolling_sharpe = pd.Series(strategy_returns).rolling(63).apply(
    lambda x: x.mean() / x.std() * np.sqrt(252) if x.std() > 0 else 0
)
axes[1].plot(rolling_sharpe.values)
axes[1].axhline(y=0, color='r', linestyle='--')
axes[1].set_xlabel('Days')
axes[1].set_ylabel('Rolling Sharpe (63d)')
axes[1].set_title('Strategy Stability')

plt.tight_layout()
plt.show()

---

## Interview Questions

### Backtesting
1. What is lookahead bias and how do you prevent it?
2. Why might a strategy work in backtest but fail live?
3. How do you account for survivorship bias?

### Transaction Costs
1. What factors determine transaction costs?
2. How would you estimate market impact for large orders?
3. When do transaction costs dominate strategy returns?

### Validation
1. Why is K-Fold CV wrong for time series?
2. How do you choose the embargo/gap period?
3. What is purging in ML for finance?

---

## Key Takeaways

| Topic | Key Point |
|-------|----------|
| Metrics | Sharpe, Sortino, Max DD - know the limitations |
| Costs | 10+ bps can kill a strategy; model realistically |
| Validation | Walk-forward, never shuffle time series |
| Biases | Lookahead, survivorship, overfitting |